# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core import Workspace, Experiment

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
ws = Workspace.from_config()

print('WORKSPACE DETAILS:\nWorkspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

# choose a name for experiment
experiment_name = 'Heart_Disease_Kaggle_AutoML'

experiment=Experiment(ws, experiment_name)
print("Experiment created")

run = experiment.start_logging()

WORKSPACE DETAILS:
Workspace name: quick-starts-ws-136719
Azure region: southcentralus
Subscription id: 5a4ab2ba-6c51-4805-8155-58759ad589d8
Resource group: aml-quickstarts-136719
Experiment created


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# Creating a compute cluster
# vm_size = "Standard_D2_V2"
# max_nodes = 4.

cpu_cluster_name = "cpu-cluster"

# to check whether the compute cluster exists already or not
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Existing compute target found... Using it")

except ComputeTargetException:
    print("Creating new Compute Target... " + cpu_cluster_name)
    provisioning_compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, provisioning_compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Existing compute target found... Using it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Exploring the Dataset

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

url_path = "https://raw.githubusercontent.com/bharati-21/AZMLND-Capstone-Project/master/files/heart.csv"
ds = TabularDatasetFactory.from_delimited_files(path=url_path)

In [8]:
ds_dataframe = ds.to_pandas_dataframe()
ds_dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [26]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x.shape, y.shape

((303, 13), (303,))

In [27]:
import os
import pandas as pd
path = "./Data"
try:
    os.makedirs(path, exist_ok=True)
    print("Dicrectory 'Data' created...")
except OSError:
    print("Directory 'Data' cannot be created...")

Dicrectory 'Data' created...


In [32]:
clean_df = x.copy(deep=True)
clean_df['target'] = y
clean_df.to_csv('./Data/clean_data.csv')
x.shape, y.shape, clean_df.shape

((303, 13), (303,), (303, 14))

In [29]:
# split data into test and train sets

from sklearn.model_selection import train_test_split
import pandas as pd

#spliting the data
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.3)

train_df = pd.concat([train_x, train_y], axis=1)
test_df = pd.concat([test_x, test_y], axis=1)

print(train_df.shape)
print(test_df.shape)

# saving test and train data locally
train_df.to_csv("./Data/train_data.csv")
test_df.to_csv("./Data/test_data.csv")

(212, 14)
(91, 14)


In [30]:
deafult_store = ws.get_default_datastore()
deafult_store.upload(src_dir="./Data", target_path="./Data", overwrite=True)

Uploading an estimated of 3 files
Uploading ./Data/clean_data.csv
Uploaded ./Data/clean_data.csv, 1 files out of an estimated total of 3
Uploading ./Data/test_data.csv
Uploaded ./Data/test_data.csv, 2 files out of an estimated total of 3
Uploading ./Data/train_data.csv
Uploaded ./Data/train_data.csv, 3 files out of an estimated total of 3
Uploaded 3 files


$AZUREML_DATAREFERENCE_fd1215697fb0430ea62ce1ff09f5893d

In [19]:
from azureml.core import Dataset

clean_data = Dataset.Tabular.from_delimited_files(path=[(deafult_store, ("./Data/clean_data.csv"))])
train_data = Dataset.Tabular.from_delimited_files(path=[(deafult_store, ("./Data/train_data.csv"))])
test_data = Dataset.Tabular.from_delimited_files(path=[(deafult_store, ("./Data/test_data.csv"))])

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [21]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name="target",
    n_cross_validations=5,
    compute_target = cpu_cluster)

print("AutoML config created.")

AutoML config created.


In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service